In [1]:
import os
import glob
import pandas as pd
import re

In [2]:
conjunto_dados = pd.DataFrame()

SPEAKER_REGEX = re.compile(r'[A|O] Sr\.ª?( [\w\-,]+)+([\-\w]+)?( \([\w\s\.,\-]+\))*(?=:)')
MULTILINE_SPEAKER_REGEX = re.compile("^" + SPEAKER_REGEX.pattern, flags=re.M) 

for file in os.scandir("./discursos"):
    speech_by_speaker = []

    with open(file, encoding="utf8") as fp:
        content = fp.read()

        #1) remover datas espalhadas, páginas e afins espalhadas pelos documentos (por causa da coversao PDF para TXT)

        datas_texto = re.compile("\d{1,2} [A-Z]+ [A-Z]+ [A-Z]+ \d{4} (\n\n\d+)?")
        datas_texto.sub(" ", content)

        series_numeros = re.compile("[A-Z]+ SÉRIE \W NÚMERO \d+ (\n+\d+)?")
        series_numeros.sub(" ", content)

        #2) criar dicionario de intervencoes para criar dataframe

        #2.1) identificar os intervenientes e intervencoes correspondentes

        deputadxs = MULTILINE_SPEAKER_REGEX.finditer(content) 
        deputadxs = [match.group() for match in deputadxs]
        deputadxs = list(set(deputadxs))

        #deputadxs = [deputadxs[5]]
        #print(deputadxs)

        content_on_one_line = ''.join(content.splitlines())

        for speaker in deputadxs:
            content_split_by_speaker = content_on_one_line.split(speaker + ':')
            # Get rid of everything before they speak
            content_split_by_speaker.pop(0)

            # what_they_say = [speech for speech in speaker_regex]
            # for each line in content_split_by_speaker
            # use the speaker regex to find the next speaker and get everything before that
            for line in content_split_by_speaker:
                start_of_next_speaker = SPEAKER_REGEX.finditer(line)

                # either an index or None
                # if it's None, either:
                # - the regex isn't accurate enough; or
                # - the speaker is the only person to speak]
                next_speaker_indexes = [match.start() for match in start_of_next_speaker]
                next_speaker_index = next_speaker_indexes[0] if len(next_speaker_indexes) > 0 else None

                speech = line[0:next_speaker_index] if next_speaker_index is not None else line
                speech_by_speaker.append((speaker, speech))

                # if "):" in speech:
                #     print(speech)
        
        df = pd.DataFrame.from_records(speech_by_speaker, columns=["speaker", "speech"])#.reset_index()
        df['filename'] = os.path.basename(file)
        #print(df)
                
        conjunto_dados = pd.concat([conjunto_dados, df])
            
#conjunto_dados#.to_csv("deputados.csv")#, index=False)
    

In [3]:
conjunto_dados.head(3)

,speaker,speech,filename
0,O Sr. Ferro Rodrigues (PS),"— Sr. Presidente, o Grupo Parlamentar do PS i...",darl13sl01n001.txt
1,A Sr.ª Teresa Leal Coelho (PSD),— Muito bem!,darl13sl01n001.txt
2,O Sr. Nuno Magalhães (CDS-PP),"— Peço a palavra, Sr. Presidente.",darl13sl01n001.txt


In [4]:
conjunto_dados["speech"] = [string.lstrip("— ") for string in conjunto_dados["speech"]]


partidos_raw = conjunto_dados["speaker"].str.split("(", n=1, expand=True)
partidos_raw.columns = ["speaker", "partido"]
partidos_raw["partido"] = partidos_raw["partido"].str.rstrip(")")
partidos_raw["speech"] = conjunto_dados["speech"]
partidos_raw["filename"] = conjunto_dados["filename"]
partidos_raw["speaker"] = partidos_raw["speaker"].str.replace(r"[A|O] Sr\.ª? ", "")
partidos_raw["speaker"] = partidos_raw["speaker"].str.rstrip(" ")
#conjunto_dados = conjunto_dados.merge(partidos_raw, how="right", on="speaker") it was breaking (maybe memory issues?)

#partidos_raw.to_csv("deputados.csv", index=False)

<ipython-input-4-104e0e56ff55>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  partidos_raw["speaker"] = partidos_raw["speaker"].str.replace(r"[A|O] Sr\.ª? ", "")


In [5]:
partidos_raw['number']=[ii[11:14] for ii in partidos_raw.filename]
partidos_raw['session']=[ii[8:10] for ii in partidos_raw.filename]
partidos_raw['legislatura']=[ii[4:6] for ii in partidos_raw.filename]
partidos_raw['speaker'] = partidos_raw['speaker'].str.lower()
partidos_raw.head()

,speaker,partido,speech,filename,number,session,legislatura
0,ferro rodrigues,PS,"Sr. Presidente, o Grupo Parlamentar do PS indi...",darl13sl01n001.txt,001,01,13
1,teresa leal coelho,PSD,Muito bem!,darl13sl01n001.txt,001,01,13
2,nuno magalhães,CDS-PP,"Peço a palavra, Sr. Presidente.",darl13sl01n001.txt,001,01,13
3,nuno magalhães,CDS-PP,"Sr. Presidente, queria apenas esclarecer que n...",darl13sl01n001.txt,001,01,13
4,nuno magalhães,CDS-PP,"Sr. Presidente, Sr.as e Srs. Deputados: Não po...",darl13sl01n001.txt,001,01,13


In [6]:
deputies=pd.read_csv('deputies_2015_now.csv')

In [7]:
deputies=deputies[['nome', 'circulo_eleitoral', 'partido', 'legislatura']]
deputies['nome'] = deputies['nome'].str.lower()
deputies.columns=['speaker', 'circulo_eleitoral', 'partido', 'legislatura']
deputies

,speaker,circulo_eleitoral,partido,legislatura
0,carlos matias,Santarém,BE,13
1,catarina martins,Porto,BE,13
2,domicilia costa,Porto,BE,13
3,ernesto ferraz,Madeira,BE,13
4,fernando manuel barbosa,Porto,BE,13
...,...,...,...,...
570,rui silva,Braga,PSD,14
571,sandra pereira,Lisboa,PSD,14
572,sara madruga da costa,Madeira,PSD,14
573,sérgio marques,Madeira,PSD,14


In [8]:
dd=partidos_raw.merge(deputies[['speaker', 'partido']], on='speaker', how='left')
speech_selected=dd[~dd.partido_y.isna()]

In [9]:
speech_selected=speech_selected[['speaker', 'partido_x', 'speech', 'filename', 'number', 'session',
       'legislatura']]
speech_selected.columns=['speaker', 'party', 'speech', 'filename', 'number', 'session',
       'legislatura']

In [10]:
speech_selected.head()

,speaker,party,speech,filename,number,session,legislatura
1,teresa leal coelho,PSD,Muito bem!,darl13sl01n001.txt,001,01,13
2,nuno magalhães,CDS-PP,"Peço a palavra, Sr. Presidente.",darl13sl01n001.txt,001,01,13
3,nuno magalhães,CDS-PP,"Sr. Presidente, queria apenas esclarecer que n...",darl13sl01n001.txt,001,01,13
4,nuno magalhães,CDS-PP,"Sr. Presidente, Sr.as e Srs. Deputados: Não po...",darl13sl01n001.txt,001,01,13
5,nuno magalhães,CDS-PP,O Sr. Presidente falou do ex-Presidente Jaime ...,darl13sl01n001.txt,001,01,13


In [11]:
speech_selected[speech_selected.legislatura=='13'].to_csv('output/speech_selected_13.csv')
speech_selected[speech_selected.legislatura=='14'].to_csv('output/speech_selected_14.csv')